# Optimization Project

In [2]:
import pandas as pd
import numpy as np
import datetime
from patsy import dmatrices
import statsmodels.api as sm
import matplotlib.pyplot as plt

/Users/douzi/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [ ]:
#Get weather dataset
from wwo_hist import retrieve_hist_data
import osfrequency = 1
start_date = '01-JAN-2014'
end_date = '30-SEP-2019'
api_key = '2ee8614f2f254803ac962041190412'
location_list = ['chicago']
hist_weather_data = retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                export_csv = True,
                                store_df = True)

In [2]:
#import arriving dataset
arrive = pd.read_csv('~/Desktop/op/optimization files_arriving.csv')

In [3]:
#import leaving dataset
leave = pd.read_csv('~/Desktop/op/optimization files_leaving.csv')

In [4]:
#import weather dataset
weather = pd.read_csv('~/Desktop/op/weather.csv')

In [5]:
#add date column
leave['date'] = pd.to_datetime(leave[['start_year', 'start_month', 'start_day','start_hour']].rename(columns={'start_year': 'year', 'start_month': 'month', 'start_day': 'day','start_hour':'hour'}))
arrive['date'] = pd.to_datetime(arrive[['end_year', 'end_month', 'end_day','end_hour']].rename(columns={'end_year': 'year', 'end_month': 'month', 'end_day': 'day','end_hour':'hour'}))

In [6]:
#drop unnecessary columns
leav = leave.drop(['Unnamed: 0','X','station_address'], axis=1)

In [7]:
weat = weather[['tempC','visibility','date_time']]
weat = weat.rename({'date_time': 'date'}, axis=1)

In [8]:
weat['date'] = pd.to_datetime(weat['date'])
weat['start_year'] = weat['date'].dt.year
weat['start_month'] = weat['date'].dt.month
weat['start_day'] = weat['date'].dt.day
weat['start_hour'] = weat['date'].dt.hour

In [9]:
#merge leave dataset with weather
leaving = pd.merge(leav, weat , on=['start_year', 'start_month','start_day','start_hour'])

In [10]:
leaving = leaving.drop(['date_y'], axis=1)
leaving = leaving.rename({'date_x': 'date'}, axis=1)

In [11]:
#add day of week colmun
leaving['day_of_week'] = leaving['date'].dt.weekday_name

##### add neigbourhood column

In [12]:
import pandas as pd
import reverse_geocode
import warnings
warnings.filterwarnings('ignore')

In [13]:
leaving = leaving.dropna()

In [14]:
# get coordinates
coords = leaving[["station_lat", "station_lon"]].apply(tuple, axis=1).tolist()

In [15]:
# Convert coordinates to neighborhoods using reverse_geocode
rg_result = reverse_geocode.search(coords)

In [16]:
neighborhoods = [x['city'] for x in rg_result]

In [17]:
leaving["neighborhoods"] = neighborhoods

In [18]:
leaving

,station_id,station_name,station_docks,station_lat,station_lon,start_hour,start_day,start_month,start_year,leaving_trips,date,tempC,visibility,day_of_week,neighborhoods
1,3,Shedd Aquarium,55.0,41.867226,-87.615355,14,21,5,2019,2,2019-05-21 14:00:00,6,4,Tuesday,Near South Side
2,6,Dusable Harbor,39.0,41.886976,-87.612813,14,21,5,2019,1,2019-05-21 14:00:00,6,4,Tuesday,Chicago Loop
3,19,Loomis St & Taylor St (*),16.0,41.868968,-87.659141,14,21,5,2019,4,2019-05-21 14:00:00,6,4,Tuesday,Lower West Side
4,20,Sheffield Ave & Kingsbury St,15.0,41.910522,-87.653106,14,21,5,2019,1,2019-05-21 14:00:00,6,4,Tuesday,Lincoln Park
5,22,May St & Taylor St,15.0,41.869482,-87.655486,14,21,5,2019,2,2019-05-21 14:00:00,6,4,Tuesday,Lower West Side
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1846111,493,Western Ave & Roscoe St,11.0,41.943034,-87.687288,2,18,1,2018,1,2018-01-18 02:00:00,-7,10,Thursday,North Center
1846112,493,Western Ave & Roscoe St,11.0,41.943034,-87.687288,2,1,2,2018,1,2018-02-01 02:00:00,1,10,Thursday,North Center
1846113,493,Western Ave & Roscoe St,11.0,41.943034,-87.687288,3,12,2,2018,1,2018-02-12 03:00:00,-13,10,Monday,North Center
1846114,517,Clark St & Jarvis Ave,11.0,42.015963,-87.675005,4,9,2,2019,1,2019-02-09 04:00:00,-13,19,Saturday,Rogers Park


In [112]:
#neig = leaving['station_id','neighborhoods']

In [113]:
#df = leaving.drop_duplicates(subset=['station_id'])

In [116]:
#df_neigbourhood = df[['station_id','neighborhoods']]

In [118]:
#df_neigbourhood

,station_id,neighborhoods
1,3,Near South Side
2,6,Chicago Loop
3,19,Lower West Side
4,20,Lincoln Park
5,22,Lower West Side
...,...,...
578000,652,Greater Grand Crossing
580945,649,Englewood
585194,648,Englewood
672226,647,Englewood


In [121]:
#station_hourly_supply_demand_updated= pd.merge(hourly_df, df_neigbourhood , on=['station_id'])

In [126]:
#station_hourly_supply_demand_updated.to_csv('station_hourly_supply_demand_updated.csv')

In [19]:
leaving = leaving.drop(['station_lat','station_lon'], axis=1)

##### Arrive 

In [20]:
#drop unnecessary columns
arriv = arrive.drop(['Unnamed: 0','X','station_address'], axis=1)

In [21]:
weat2 = weather[['tempC','visibility','date_time']]
weat2 = weat2.rename({'date_time': 'date'}, axis=1)

In [22]:
weat2['date'] = pd.to_datetime(weat2['date'])
weat2['end_year'] = weat2['date'].dt.year
weat2['end_month'] = weat2['date'].dt.month
weat2['end_day'] = weat2['date'].dt.day
weat2['end_hour'] = weat2['date'].dt.hour

In [23]:
#merge leave dataset with weather
arriving2 = pd.merge(arriv, weat2 , on=['end_year', 'end_month','end_day','end_hour'])

In [24]:
arriving2 = arriving2.drop(['date_y'], axis=1)
arriving2 = arriving2.rename({'date_x': 'date'}, axis=1)

In [25]:
#add day of week colmun
arriving2['day_of_week'] = arriving2['date'].dt.weekday_name

In [26]:
arriving2

,station_id,station_name,station_docks,station_lat,station_lon,end_hour,end_day,end_month,end_year,arriving_trips,date,tempC,visibility,day_of_week
0,1,NaN,NaN,NaN,NaN,14,21,5,2019,1,2019-05-21 14:00:00,6,4,Tuesday
1,5,State St & Harrison St,23.0,41.874053,-87.627716,14,21,5,2019,1,2019-05-21 14:00:00,6,4,Tuesday
2,6,Dusable Harbor,39.0,41.886976,-87.612813,14,21,5,2019,1,2019-05-21 14:00:00,6,4,Tuesday
3,7,Field Blvd & South Water St,19.0,41.886349,-87.617517,14,21,5,2019,1,2019-05-21 14:00:00,6,4,Tuesday
4,13,Wilton Ave & Diversey Pkwy,35.0,41.932418,-87.652705,14,21,5,2019,1,2019-05-21 14:00:00,6,4,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835842,493,Western Ave & Roscoe St,11.0,41.943030,-87.687290,4,24,2,2018,1,2018-02-24 04:00:00,1,10,Saturday
1835843,489,Drake Ave & Addison St,15.0,41.947330,-87.717580,3,6,11,2018,1,2018-11-06 03:00:00,9,9,Tuesday
1835844,491,Talman Ave & Addison St,15.0,41.946640,-87.694610,2,5,11,2018,1,2018-11-05 02:00:00,8,10,Monday
1835845,624,Dearborn St & Van Buren St (*),16.0,41.876270,-87.629150,0,26,11,2018,1,2018-11-26 00:00:00,1,10,Monday


In [27]:
arriving2 = arriving2.dropna()

In [28]:
# get coordinates
coords2 = arriving2[["station_lat", "station_lon"]].apply(tuple, axis=1).tolist()

In [29]:
# Convert coordinates to neighborhoods using reverse_geocode
rg_result2 = reverse_geocode.search(coords2)

In [30]:
neighborhoods2 = [x['city'] for x in rg_result2]

In [31]:
arriving2["neighborhoods"] = neighborhoods2

In [32]:
arriving2

,station_id,station_name,station_docks,station_lat,station_lon,end_hour,end_day,end_month,end_year,arriving_trips,date,tempC,visibility,day_of_week,neighborhoods
1,5,State St & Harrison St,23.0,41.874053,-87.627716,14,21,5,2019,1,2019-05-21 14:00:00,6,4,Tuesday,Chicago Loop
2,6,Dusable Harbor,39.0,41.886976,-87.612813,14,21,5,2019,1,2019-05-21 14:00:00,6,4,Tuesday,Chicago Loop
3,7,Field Blvd & South Water St,19.0,41.886349,-87.617517,14,21,5,2019,1,2019-05-21 14:00:00,6,4,Tuesday,Chicago Loop
4,13,Wilton Ave & Diversey Pkwy,35.0,41.932418,-87.652705,14,21,5,2019,1,2019-05-21 14:00:00,6,4,Tuesday,Lincoln Park
5,15,Racine Ave & 18th St,15.0,41.858166,-87.656495,14,21,5,2019,2,2019-05-21 14:00:00,6,4,Tuesday,Lower West Side
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835842,493,Western Ave & Roscoe St,11.0,41.943030,-87.687290,4,24,2,2018,1,2018-02-24 04:00:00,1,10,Saturday,North Center
1835843,489,Drake Ave & Addison St,15.0,41.947330,-87.717580,3,6,11,2018,1,2018-11-06 03:00:00,9,9,Tuesday,Avondale
1835844,491,Talman Ave & Addison St,15.0,41.946640,-87.694610,2,5,11,2018,1,2018-11-05 02:00:00,8,10,Monday,North Center
1835845,624,Dearborn St & Van Buren St (*),16.0,41.876270,-87.629150,0,26,11,2018,1,2018-11-26 00:00:00,1,10,Monday,Chicago Loop


In [33]:
arriving2.loc[arriving2['day_of_week'].isin(['Sunday', 'Saturday']), 'day_of_week' ] = 'Weekend'  

In [34]:
arriving2.loc[arriving2['day_of_week']!='Weekend', 'day_of_week' ] = 'Weekday'

In [35]:
leaving.loc[leaving['day_of_week'].isin(['Sunday', 'Saturday']), 'day_of_week' ] = 'Weekend' 

In [36]:
leaving.loc[leaving['day_of_week']!='Weekend', 'day_of_week' ] = 'Weekday'

In [58]:
#leaving_loop = leaving.loc[leaving['neighborhoods'].isin(['Chicago Loop','Uptown'])]

In [60]:
#arriving_loop = arriving2.loc[arriving2['neighborhoods'].isin(['Chicago Loop','Uptown'])]

In [68]:
#leaving_chicagoloop = leaving.loc[leaving['neighborhoods'].isin(['Chicago Loop'])]

In [69]:
#leaving_Uptown = leaving.loc[leaving['neighborhoods'].isin(['Uptown'])]

In [71]:
#x= leaving_chicagoloop['station_id'].value_counts()

In [73]:
#y= leaving_Uptown['station_id'].value_counts()

In [37]:
#leaving_divvy = leaving.loc[leaving['station_id'].isin(['46','195','56','631','97','191','100','301','2','626','6','359','365','89','74','35','47','90','75','112','268','20','331','186','59','72','217','211','85','181'])]

In [38]:
#arriving_divvy = arriving2.loc[arriving2['station_id'].isin(['46','195','56','631','97','191','100','301','2','626','6','359','365','89','74','35','47','90','75','112','268','20','331','186','59','72','217','211','85','181'])]

In [80]:
#arriving_divvy = arriving2.loc[arriving2['station_id'].isin(['240','303','114','232','254','304','312','231','256','165','293','326','306','234','253','77','133','195','192','56','199','24','44','66','26','43','81','198','181','71'])]

In [55]:
leaving.to_csv('leaving_structuredall.csv')

In [56]:
arriving2.to_csv('arriving_structuredall.csv')

In [45]:
arriving_divvy.to_csv('arriving_structuredall.csv')

In [46]:
leaving_divvy.to_csv('leaving_structuredall.csv')